In [ ]:
import pandas
import requests
import re
import time
from pathlib import Path
d = Path().resolve().parent.parent.parent
csv = pandas.read_csv(str(d) + "/data/original/authors.csv")

#Because Google sometimes blocks us out, this variable is used to specify in which author it should start.
i=0


for j in range(i,len(csv)):
    if(i<8800):
        new_name = csv['name'][j].replace(" ", "+")
        name_separated = csv['name'][j].split(" ")
        ident = csv['id'][j]
        
        url = "https://www.google.com/search?q="
        query = new_name + "+google+scholar"
        source = url+query

        #Get info from the page
        r = requests.get(source)

        #print(source)
        #Get the content of the page as a string
        toStr = "".join(map(chr, r.content))

        #Find ?user inside that to obtain the id in Google Scholar
        position = toStr.find('?user')


        #Obtain the code
        code = toStr[position+6:position+18]
        #Build query with id obtained in the step before
        urlGS = "https://scholar.google.com/citations?user="
        sourceGS = urlGS+code

        #request info from that url
        rGS = requests.get(sourceGS)

        #Get the content of the page as a string
        toStrGS = "".join(map(chr, rGS.content))
        
        #Find the name and last name and compare
        namePositionGS = toStrGS.find('<div id="gsc_prf_in">')
        nameGS = toStrGS[namePositionGS+21:namePositionGS+80]
        fullNameGS = ""
        fullNameGSbool = 1
        for letter in nameGS:
            if(letter!='<'):
                fullNameGS+= letter
            else:
                break
        splittedNameGS = fullNameGS.split(" ") 
        if(splittedNameGS[0]==name_separated[0] and splittedNameGS[-1]==name_separated[-1]):
            #We looked for a link of an organization, that is why if there is not a link, this one can not be identified.
            positionuniGS = toStrGS.find('/citations?view_op=view_org')
            codeuniGS = toStrGS[positionuniGS+1:positionuniGS+200]
            uniGS = ""
            uniGSbool = 0
            for letter in codeuniGS:
                if(uniGSbool==1 and letter=='<'):
                    break
                if(uniGSbool==1):
                    uniGS+=letter
                if(uniGSbool==0 and letter=='>'):
                    uniGSbool=1

            uni = uniGS
        else:
            uni = "Not in Google Scholar"
        print(str(ident) + "; " + fullNameGS + "; " + uni)
        i+=1
        if(i%300==0):
            time.sleep(300)
    else:
        print("Done")
        break